In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')

print("="*80)
print("ENHANCED BACKTESTING FRAMEWORK")
print("="*80)

In [ ]:
# Backtest Configuration
CONFIG = {
    "version": "2.0.0",
    "random_seed": RANDOM_SEED,
    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    
    # Transaction Costs
    "transaction_costs": {
        "commission_bps": 5,        # Commission in basis points
        "slippage_bps": 3,          # Slippage in basis points
        "market_impact_bps": 2      # Market impact in basis points
    },
    
    # Risk Constraints
    "risk_constraints": {
        "max_drawdown_pct": 0.20,      # Maximum 20% drawdown
        "max_volatility": 0.15,         # Maximum 15% volatility (annualized)
        "max_sector_exposure": 0.35,    # Maximum 35% per sector
        "max_asset_exposure": 0.10,     # Maximum 10% per asset
        "leverage_limit": 1.0           # No leverage
    },
    
    # Position Sizing
    "position_sizing": {
        "method": "volatility_targeted",  # Options: equal_weight, volatility_targeted, risk_parity
        "target_volatility": 0.10,         # Target 10% annualized volatility
        "rebalance_frequency": "weekly",   # Rebalancing frequency
        "min_position_size": 0.01,         # Minimum 1% position
        "max_position_size": 0.10          # Maximum 10% position
    },
    
    # Walk-Forward Validation
    "walk_forward": {
        "train_size": 0.7,         # 70% training window
        "test_size": 0.3,          # 30% testing window
        "overlap": 0.5,            # 50% overlap between windows
        "min_train_periods": 50    # Minimum periods for training
    },
    
    # Strategy Parameters
    "strategies": {
        "momentum": {
            "lookback_period": 1,
            "holding_period": 1,
            "threshold": 0.0
        },
        "mean_reversion": {
            "lookback_period": 1,
            "mean_window": 20,
            "std_threshold": 1.5
        }
    }
}

# Save configuration for reproducibility
with open('backtest_config.json', 'w') as f:
    json.dump(CONFIG, f, indent=4)

print("\n📋 Configuration saved: backtest_config.json")
print(f"🔢 Random Seed: {RANDOM_SEED}")
print(f"⏰ Timestamp: {CONFIG['timestamp']}")
print(f"📊 Version: {CONFIG['version']}")

In [ ]:
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

# Load trades and instruments data
trades = pd.read_csv("../Data/project2_trading.csv")
instruments = pd.read_csv("../Data/project2_instruments.csv")

print(f"\n✅ Loaded {len(trades)} trades")
print(f"✅ Loaded {len(instruments)} instruments")

print("\nTrades sample:")
print(trades.head())

# Create returns per instrument
returns = trades.pivot_table(
    index="timestamp", 
    columns="instrument_id", 
    values="pnl_usd"
).pct_change().dropna()

print(f"\n📊 Returns matrix shape: {returns.shape}")
print(f"📅 Date range: {returns.index.min()} to {returns.index.max()}")

# Get instrument metadata
instrument_map = instruments.set_index('instrument_id')[['instrument_name', 'sector', 'asset_class']].to_dict('index')


In [ ]:
def calculate_transaction_costs(signals, returns, config):
    """
    Calculate comprehensive transaction costs including:
    - Commission costs
    - Slippage costs
    - Market impact costs
    """
    commission_bps = config['transaction_costs']['commission_bps']
    slippage_bps = config['transaction_costs']['slippage_bps']
    impact_bps = config['transaction_costs']['market_impact_bps']
    
    total_cost_bps = commission_bps + slippage_bps + impact_bps
    
    # Calculate turnover (position changes)
    turnover = abs(signals.diff()).fillna(0)
    
    # Transaction costs proportional to turnover
    tc_per_asset = (total_cost_bps / 10000) * turnover * abs(returns)
    tc_total = tc_per_asset.sum(axis=1)
    
    return tc_total, tc_per_asset

In [ ]:
def volatility_targeted_sizing(returns, signals, target_vol=0.10):
    """
    Size positions based on volatility targeting
    """
    # Calculate rolling volatility (annualized)
    rolling_vol = returns.rolling(window=20).std() * np.sqrt(252)
    
    # Scale positions inversely to volatility
    vol_scalar = target_vol / (rolling_vol + 1e-8)
    vol_scalar = vol_scalar.clip(0.5, 2.0)  # Limit scaling between 0.5x and 2x
    
    # Apply to signals
    sized_signals = signals * vol_scalar
    
    return sized_signals

def apply_position_limits(positions, config):
    """
    Apply position size limits
    """
    max_pos = config['position_sizing']['max_position_size']
    min_pos = config['position_sizing']['min_position_size']
    
    # Clip positions
    positions_clipped = positions.clip(-max_pos, max_pos)
    
    # Set small positions to zero
    positions_clipped[abs(positions_clipped) < min_pos] = 0
    
    # Normalize to ensure leverage limit
    leverage_limit = config['risk_constraints']['leverage_limit']
    total_exposure = abs(positions_clipped).sum(axis=1)
    
    # Scale down if over leverage limit
    scale_factor = np.minimum(1.0, leverage_limit / (total_exposure + 1e-8))
    positions_final = positions_clipped.multiply(scale_factor, axis=0)
    
    return positions_final
def max_drawdown(series):
    """Calculate maximum drawdown"""
    roll_max = series.cummax()
    drawdown = (series - roll_max) / (roll_max + 1e-8)
    return drawdown.min()


In [ ]:
def check_risk_constraints(cum_pnl, daily_pnl, positions, config, instrument_map):
    """
    Check and enforce risk constraints
    """
    constraints_violated = []
    
    # 1. Max Drawdown Check
    max_dd = max_drawdown(cum_pnl)
    max_dd_limit = config['risk_constraints']['max_drawdown_pct']
    if abs(max_dd) > max_dd_limit:
        constraints_violated.append(f"Max Drawdown: {max_dd:.2%} exceeds limit {max_dd_limit:.2%}")
    
    # 2. Volatility Check
    vol = daily_pnl.std() * np.sqrt(252)
    max_vol = config['risk_constraints']['max_volatility']
    if vol > max_vol:
        constraints_violated.append(f"Volatility: {vol:.2%} exceeds limit {max_vol:.2%}")
    
    # 3. Sector Exposure Check
    max_sector_exp = config['risk_constraints']['max_sector_exposure']
    for timestamp in positions.index[-10:]:  # Check recent positions
        pos = positions.loc[timestamp]
        for sector in set([instrument_map.get(inst, {}).get('sector', 'Unknown') for inst in pos.index]):
            sector_instruments = [inst for inst in pos.index 
                                if instrument_map.get(inst, {}).get('sector') == sector]
            sector_exposure = abs(pos[sector_instruments]).sum()
            if sector_exposure > max_sector_exp:
                constraints_violated.append(
                    f"Sector {sector} exposure: {sector_exposure:.2%} exceeds limit {max_sector_exp:.2%}"
                )
                break
    
    # 4. Asset Exposure Check
    max_asset_exp = config['risk_constraints']['max_asset_exposure']
    max_position = abs(positions).max().max()
    if max_position > max_asset_exp:
        constraints_violated.append(f"Max asset exposure: {max_position:.2%} exceeds limit {max_asset_exp:.2%}")
    
    return constraints_violated

In [ ]:
def momentum_strategy(ret_series, config):
    """
    Enhanced Momentum Strategy:
    - Go long if recent returns are positive
    - Go short if recent returns are negative
    - Includes lookback period and threshold
    """
    lookback = config['strategies']['momentum']['lookback_period']
    threshold = config['strategies']['momentum']['threshold']
    
    signals = np.where(ret_series.shift(lookback) > threshold, 1, -1)
    return signals

def mean_reversion_strategy(ret_series, config):
    """
    Enhanced Mean Reversion Strategy:
    - Go long if price below mean - threshold*std
    - Go short if price above mean + threshold*std
    - Uses rolling statistics
    """
    mean_window = config['strategies']['mean_reversion']['mean_window']
    std_threshold = config['strategies']['mean_reversion']['std_threshold']
    
    rolling_mean = ret_series.rolling(window=mean_window).mean()
    rolling_std = ret_series.rolling(window=mean_window).std()
    
    z_score = (ret_series - rolling_mean) / (rolling_std + 1e-8)
    
    # Mean reversion: buy when oversold, sell when overbought
    signals = np.where(z_score < -std_threshold, 1,
                      np.where(z_score > std_threshold, -1, 0))
    
    return signals

In [ ]:
def backtest_strategy(returns, strategy_func, config, instrument_map):
    """
    Enhanced backtesting with full cost modeling and risk constraints
    """
    print(f"\n🔄 Running backtest...")
    
    # Generate raw signals
    signals = returns.apply(lambda x: strategy_func(x, config))
    signals = signals.fillna(0)
    
    # Apply position sizing
    if config['position_sizing']['method'] == 'volatility_targeted':
        signals = volatility_targeted_sizing(
            returns, signals, 
            config['position_sizing']['target_volatility']
        )
    
    # Apply position limits
    positions = apply_position_limits(signals, config)
    
    # Calculate P&L before costs
    pnl_gross = (positions.shift(1) * returns).sum(axis=1)
    
    # Calculate transaction costs
    tc_total, tc_per_asset = calculate_transaction_costs(positions, returns, config)
    
    # Net P&L after costs
    pnl_net = pnl_gross - tc_total
    
    # Cumulative P&L
    cum_pnl = pnl_net.cumsum()
    
    # Check risk constraints
    constraints_violated = check_risk_constraints(
        cum_pnl, pnl_net, positions, config, instrument_map
    )
    
    return {
        'cum_pnl': cum_pnl,
        'daily_pnl': pnl_net,
        'positions': positions,
        'transaction_costs': tc_total,
        'gross_pnl': pnl_gross,
        'constraints_violated': constraints_violated
    }




In [ ]:
print("\n" + "="*80)
print("STANDARD BACKTEST")
print("="*80)

# Run Momentum Strategy
print("\n📈 Testing Momentum Strategy...")
momentum_results = backtest_strategy(returns, momentum_strategy, CONFIG, instrument_map)

# Run Mean Reversion Strategy
print("📉 Testing Mean Reversion Strategy...")
mr_results = backtest_strategy(returns, mean_reversion_strategy, CONFIG, instrument_map)

# Plot results
plt.figure(figsize=(14, 6))
plt.plot(momentum_results['cum_pnl'].index, momentum_results['cum_pnl'].values, 
         label="Momentum Strategy", linewidth=2)
plt.plot(mr_results['cum_pnl'].index, mr_results['cum_pnl'].values, 
         label="Mean Reversion Strategy", linewidth=2)
plt.legend(fontsize=12)
plt.title("Strategy Backtest Results (With Transaction Costs & Risk Constraints)", fontsize=14, fontweight='bold')
plt.xlabel("Time", fontsize=12)
plt.ylabel("Cumulative P&L", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("backtest_results_plot.png", dpi=300, bbox_inches='tight')
print("\n📊 Plot saved: backtest_results_plot.png")
plt.show()

In [ ]:
def max_drawdown(series):
    """Calculate maximum drawdown"""
    roll_max = series.cummax()
    drawdown = (series - roll_max) / (roll_max + 1e-8)
    return drawdown.min()

def calculate_metrics(results):
    """Calculate comprehensive performance metrics"""
    cum_pnl = results['cum_pnl']
    daily_pnl = results['daily_pnl']
    
    metrics = {
        'Total Return': cum_pnl.iloc[-1],
        'Volatility': daily_pnl.std() * np.sqrt(252),  # Annualized
        'Sharpe': (daily_pnl.mean() / daily_pnl.std()) * np.sqrt(252),  # Annualized
        'Max Drawdown': max_drawdown(cum_pnl),
        'Win Rate': (daily_pnl > 0).sum() / len(daily_pnl),
        'Avg Transaction Cost': results['transaction_costs'].mean(),
        'Total Transaction Cost': results['transaction_costs'].sum(),
        'Gross Return': results['gross_pnl'].sum(),
        'Net Return': daily_pnl.sum(),
        'Avg Daily Return': daily_pnl.mean(),
        'Best Day': daily_pnl.max(),
        'Worst Day': daily_pnl.min()
    }
    
    return metrics

# Calculate metrics
momentum_metrics = calculate_metrics(momentum_results)
mr_metrics = calculate_metrics(mr_results)

# Create results dataframe
results = pd.DataFrame({
    "Strategy": ["Momentum", "Mean Reversion"],
    "Total Return": [momentum_metrics['Total Return'], mr_metrics['Total Return']],
    "Volatility": [momentum_metrics['Volatility'], mr_metrics['Volatility']],
    "Sharpe": [momentum_metrics['Sharpe'], mr_metrics['Sharpe']],
    "Max Drawdown": [momentum_metrics['Max Drawdown'], mr_metrics['Max Drawdown']]
})

print("\n" + "="*80)
print("BACKTEST PERFORMANCE")
print("="*80)
print(results.to_string(index=False))

# Display risk constraint violations
print("\n🚨 Risk Constraint Checks:")
if momentum_results['constraints_violated']:
    print("Momentum Strategy Violations:")
    for v in momentum_results['constraints_violated']:
        print(f"  ⚠️  {v}")
else:
    print("✅ Momentum Strategy: All constraints satisfied")

if mr_results['constraints_violated']:
    print("\nMean Reversion Strategy Violations:")
    for v in mr_results['constraints_violated']:
        print(f"  ⚠️  {v}")
else:
    print("✅ Mean Reversion Strategy: All constraints satisfied")

# Save results
results.to_csv("backtest_results.csv", index=False)
print("\n💾 Backtest results saved: backtest_results.csv")


In [ ]:
print("\n" + "="*80)
print("WALK-FORWARD VALIDATION")
print("="*80)

def walk_forward_validation(returns, strategy_func, config, instrument_map):
    """
    Walk-forward validation with rolling windows
    """
    train_pct = config['walk_forward']['train_size']
    overlap = config['walk_forward']['overlap']
    min_periods = config['walk_forward']['min_train_periods']
    
    n = len(returns)
    train_size = int(train_pct * n)
    test_size = n - train_size
    step_size = int(train_size * (1 - overlap))
    
    print(f"\n📊 Walk-Forward Configuration:")
    print(f"  Total periods: {n}")
    print(f"  Train size: {train_size} ({train_pct*100:.0f}%)")
    print(f"  Test size per window: {step_size}")
    print(f"  Overlap: {overlap*100:.0f}%")
    
    all_pnls = []
    all_positions = []
    all_costs = []
    window_count = 0
    
    for start in range(0, n - train_size, step_size):
        end_train = start + train_size
        end_test = min(end_train + step_size, n)
        
        if end_test - end_train < min_periods:
            break
        
        window_count += 1
        
        # Training and testing data
        train_data = returns.iloc[start:end_train]
        test_data = returns.iloc[end_train:end_test]
        
        # Generate signals on test data
        signals = test_data.apply(lambda x: strategy_func(x, config))
        signals = signals.fillna(0)
        
        # Apply position sizing
        if config['position_sizing']['method'] == 'volatility_targeted':
            signals = volatility_targeted_sizing(
                test_data, signals,
                config['position_sizing']['target_volatility']
            )
        
        # Apply position limits
        positions = apply_position_limits(signals, config)
        
        # Calculate P&L
        pnl_gross = (positions.shift(1) * test_data).sum(axis=1)
        tc_total, _ = calculate_transaction_costs(positions, test_data, config)
        pnl_net = pnl_gross - tc_total
        
        all_pnls.append(pnl_net)
        all_positions.append(positions)
        all_costs.append(tc_total)
    
    print(f"  ✅ Completed {window_count} walk-forward windows")
    
    # Combine results
    combined_pnl = pd.concat(all_pnls).sort_index()
    combined_positions = pd.concat(all_positions).sort_index()
    combined_costs = pd.concat(all_costs).sort_index()
    
    return {
        'cum_pnl': combined_pnl.cumsum(),
        'daily_pnl': combined_pnl,
        'positions': combined_positions,
        'transaction_costs': combined_costs,
        'gross_pnl': combined_pnl + combined_costs,
        'constraints_violated': []  # Can add constraint checks here
    }

# Run Walk-Forward Validation
print("\n📈 Walk-Forward: Momentum Strategy...")
momentum_wf_results = walk_forward_validation(returns, momentum_strategy, CONFIG, instrument_map)

print("\n📉 Walk-Forward: Mean Reversion Strategy...")
mr_wf_results = walk_forward_validation(returns, mean_reversion_strategy, CONFIG, instrument_map)

# Plot Walk-Forward results
plt.figure(figsize=(14, 6))
plt.plot(momentum_wf_results['cum_pnl'].index, momentum_wf_results['cum_pnl'].values,
         label="Momentum (Walk-Forward)", linewidth=2)
plt.plot(mr_wf_results['cum_pnl'].index, mr_wf_results['cum_pnl'].values,
         label="Mean Reversion (Walk-Forward)", linewidth=2)
plt.legend(fontsize=12)
plt.title("Walk-Forward Backtest Results", fontsize=14, fontweight='bold')
plt.xlabel("Time", fontsize=12)
plt.ylabel("Cumulative P&L", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("backtest_walkforward_plot.png", dpi=300, bbox_inches='tight')
print("\n📊 Plot saved: backtest_walkforward_plot.png")
plt.show()

# Calculate Walk-Forward metrics
momentum_wf_metrics = calculate_metrics(momentum_wf_results)
mr_wf_metrics = calculate_metrics(mr_wf_results)

# Create Walk-Forward results dataframe
results_wf = pd.DataFrame({
    "Strategy": ["Momentum (WF)", "Mean Reversion (WF)"],
    "Total Return": [momentum_wf_metrics['Total Return'], mr_wf_metrics['Total Return']],
    "Volatility": [momentum_wf_metrics['Volatility'], mr_wf_metrics['Volatility']],
    "Sharpe": [momentum_wf_metrics['Sharpe'], mr_wf_metrics['Sharpe']],
    "Max Drawdown": [momentum_wf_metrics['Max Drawdown'], mr_wf_metrics['Max Drawdown']]
})

print("\n" + "="*80)
print("WALK-FORWARD PERFORMANCE")
print("="*80)
print(results_wf.to_string(index=False))

# Save Walk-Forward results
results_wf.to_csv("backtest_results_walkforward.csv", index=False)
print("\n💾 Walk-Forward results saved: backtest_results_walkforward.csv")


In [ ]:
print("\n" + "="*80)
print("DETAILED ANALYSIS")
print("="*80)

# Comparison of Standard vs Walk-Forward
comparison = pd.DataFrame({
    'Metric': ['Total Return', 'Volatility', 'Sharpe Ratio', 'Max Drawdown'],
    'Momentum (Std)': [
        momentum_metrics['Total Return'],
        momentum_metrics['Volatility'],
        momentum_metrics['Sharpe'],
        momentum_metrics['Max Drawdown']
    ],
    'Momentum (WF)': [
        momentum_wf_metrics['Total Return'],
        momentum_wf_metrics['Volatility'],
        momentum_wf_metrics['Sharpe'],
        momentum_wf_metrics['Max Drawdown']
    ],
    'MeanRev (Std)': [
        mr_metrics['Total Return'],
        mr_metrics['Volatility'],
        mr_metrics['Sharpe'],
        mr_metrics['Max Drawdown']
    ],
    'MeanRev (WF)': [
        mr_wf_metrics['Total Return'],
        mr_wf_metrics['Volatility'],
        mr_wf_metrics['Sharpe'],
        mr_wf_metrics['Max Drawdown']
    ]
})

print("\n📊 Standard vs Walk-Forward Comparison:")
print(comparison.to_string(index=False))

# Save detailed metrics
detailed_metrics = {
    'momentum_standard': momentum_metrics,
    'momentum_walkforward': momentum_wf_metrics,
    'mean_reversion_standard': mr_metrics,
    'mean_reversion_walkforward': mr_wf_metrics
}

with open('backtest_detailed_metrics.json', 'w') as f:
    # Convert numpy types to Python types for JSON serialization
    serializable_metrics = {}
    for key, val in detailed_metrics.items():
        serializable_metrics[key] = {k: float(v) if isinstance(v, (np.integer, np.floating)) else v 
                                     for k, v in val.items()}
    json.dump(serializable_metrics, f, indent=4)

print("\n💾 Detailed metrics saved: backtest_detailed_metrics.json")

In [ ]:
print("\n" + "="*80)
print("BACKTEST SUMMARY")
print("="*80)

print(f"""
✅ Backtesting Complete

📁 Output Files Generated:
  ✓ backtest_config.json - Configuration for reproducibility
  ✓ backtest_results.csv - Standard backtest results
  ✓ backtest_results_walkforward.csv - Walk-forward validation results
  ✓ backtest_detailed_metrics.json - Comprehensive metrics
  ✓ backtest_results_plot.png - Standard backtest visualization
  ✓ backtest_walkforward_plot.png - Walk-forward visualization

🎯 Key Features Implemented:
  ✓ Momentum & Mean Reversion Strategies
  ✓ Walk-Forward Validation
  ✓ Transaction Cost Modeling (Commission, Slippage, Market Impact)
  ✓ Volatility-Targeted Position Sizing
  ✓ Risk Constraints (Max Drawdown, Volatility Caps, Sector/Asset Limits)
  ✓ Full Reproducibility (Configs, Seeds, Versions)

📊 Best Performing Strategy:
  Standard: {results.loc[results['Sharpe'].idxmax(), 'Strategy']} (Sharpe: {results['Sharpe'].max():.4f})
  Walk-Forward: {results_wf.loc[results_wf['Sharpe'].idxmax(), 'Strategy']} (Sharpe: {results_wf['Sharpe'].max():.4f})

🔢 Reproducibility:
  Version: {CONFIG['version']}
  Random Seed: {CONFIG['random_seed']}
  Timestamp: {CONFIG['timestamp']}
""")

print("="*80)
print("✅ BACKTESTING FRAMEWORK COMPLETE")
print("="*80)